# Consultas MONGODB

Nicolás Fernández y Néstor Villa

In [ ]:
# Importamos las clases y el método init_app
from models import Cliente, Producto, Compra, Proveedor, init_app
from bson import ObjectId
from datetime import datetime

# Iniciamos la conexión a la base de datos
init_app()

In [ ]:
# 1. Listado de todas las compras de un cliente
pipeline = [
    {
        "$group": {
            "_id": "$cliente.nombre",
            "compras": { "$push": { "$toString": "$_id" } }
        }
    },
    {
        "$project": {
            "nombre_cliente": "$_id",
            "compras": 1,
            "_id": 0
        }
    }
]

compras_cliente = list(Compra.aggregate(pipeline))
compras_cliente


In [ ]:
# 2. Listado de todos los proveedores para un producto
pipeline = [
    { "$unwind": "$proveedores" },
    {
        "$group": {
            "_id": "$nombre",
            "proveedores": { "$push": "$proveedores.nombre" }
        }
    },
    {
        "$project": {
            "_id": 0,
            "nombre_producto": "$_id",
            "proveedores": 1
        }
    }
]

proveedores_producto = list(Producto.aggregate(pipeline))
proveedores_producto


In [ ]:
# 3. Listado de todos los productos diferentes comprados por un cliente
pipeline = [
    { "$unwind": "$productos" },
    {
        "$group": {
            "_id": "$cliente.nombre",
            "productos": { "$addToSet": "$productos.nombre" }
        }
    },
    {
        "$project": {
            "nombre_cliente": "$_id",
            "productos": 1,
            "_id": 0
        }
    }
]

productos_cliente = list(Compra.aggregate(pipeline))
productos_cliente


In [ ]:
# 4. Listado de productos vendidos por “Modas Paqui” cuyo nombre contenga “manga corta”
pipeline = [
    { "$unwind": "$proveedores" },
    {
        "$match": {
            "proveedores.nombre": "Modas Paqui",
            "nombre": { "$regex": "manga corta", "$options": "i" }
        }
    },
    {
        "$project": {
            "_id": 0,
            "nombre_producto": "$nombre",
            "proveedor": "$proveedores.nombre"
        }
    }
]

productos_manga_corta = list(Producto.aggregate(pipeline))
productos_manga_corta


In [ ]:
# 5. Calcular el peso y volumen total de los productos comprados por un cliente un día determinado
pipeline = [
    {
        "$match": {
            "cliente.nombre": "Beatriz Gómez",
            "fecha_compra": datetime(2024, 4, 11)
        }
    },
    { "$unwind": "$productos" },
    {
        "$group": {
            "_id": None,
            "peso_total_kg": { "$sum": "$productos.peso" },
            "volumen_total_metros": {
                "$sum": {
                    "$multiply": [
                        { "$divide": [{ "$toDouble": "$productos.dimensiones.ancho" }, 100] },
                        { "$divide": [{ "$toDouble": "$productos.dimensiones.alto" }, 100] },
                        { "$divide": [{ "$toDouble": "$productos.dimensiones.profundidad" }, 100] }
                    ]
                }
            }
        }
    },
    {
        "$project": {
            "_id": 0,
            "peso_total_kg": 1,
            "volumen_total_metros": 1
        }
    }
]

peso_volumen_cliente = list(Compra.aggregate(pipeline))
peso_volumen_cliente


In [ ]:
# 6. Calcular el número medio de envíos por mes y almacén
pipeline = [
    { "$unwind": "$productos" },
    { "$unwind": "$productos.proveedores" },
    { "$unwind": "$productos.proveedores.direcciones_almacenes" },
    {
        "$group": {
            "_id": {
                "almacen": "$productos.proveedores.direcciones_almacenes.calle",
                "mes": { "$month": "$fecha_compra" },
                "año": { "$year": "$fecha_compra" }
            },
            "total_envios": { "$sum": 1 }
        }
    },
    {
        "$group": {
            "_id": "$_id.almacen",
            "promedio_envios_mensuales": { "$avg": "$total_envios" }
        }
    },
    {
        "$project": {
            "_id": 0,
            "almacen": "$_id",
            "promedio_envios_mensuales": 1
        }
    }
]

promedio_envios = list(Compra.aggregate(pipeline))
promedio_envios


In [ ]:
# 7. Listado con los tres proveedores con más volumen de facturación
pipeline = [
    { "$unwind": "$productos" },
    { "$unwind": "$productos.proveedores" },
    {
        "$group": {
            "_id": "$productos.proveedores.nombre",
            "volumen_facturacion": { "$sum": "$precio_compra" }
        }
    },
    { "$sort": { "volumen_facturacion": -1 } },
    { "$limit": 3 },
    {
        "$project": {
            "_id": 0,
            "proveedor": "$_id",
            "volumen_facturacion": 1
        }
    }
]

top_proveedores = list(Compra.aggregate(pipeline))
top_proveedores


In [ ]:
# 8. Listado de almacenes cerca de unas coordenadas determinadas (100km de distancia máxima)
pipeline = [
    {
        "$geoNear": {
            "near": { "type": "Point", "coordinates": [-9.137398, 38.710141] },
            "distanceField": "distancia",
            "maxDistance": 100 * 1000,
            "spherical": True,
            "query": { "direcciones_almacenes.location": { "$exists": True } }
        }
    },
    { "$unwind": "$direcciones_almacenes" },
    { "$sort": { "distancia": 1 } },
    {
        "$project": {
            "_id": 0,
            "nombre_proveedor": "$nombre",
            "direccion_almacen": "$direcciones_almacenes.calle",
            "ciudad": "$direcciones_almacenes.ciudad",
            "distancia": 1,
            "location": "$direcciones_almacenes.location"
        }
    }
]

almacenes_cercanos = list(Proveedor.aggregate(pipeline))
almacenes_cercanos


In [ ]:
# 9. Listado de compras dentro de un polígono cuyos vértices están definidos
pipeline = [
    {
        "$match": {
            "direccion_envio.location": {
                "$geoWithin": {
                    "$geometry": {
                        "type": "Polygon",
                        "coordinates": [[
                            [-4.3452, 40.7336], [-3.3580, 40.8174],
                            [-3.2087, 40.3119], [-4.0261, 40.1802], [-4.3452, 40.7336]
                        ]]
                    }
                }
            }
        }
    },
    {
        "$project": {
            "_id": 0,
            "compra_id": "$_id",
            "cliente": "$cliente.nombre",
            "productos": "$productos.nombre",
            "direccion_envio": "$direccion_envio",
            "location": "$direccion_envio.location"
        }
    }
]

compras_poligono = list(Compra.aggregate(pipeline))
compras_poligono


In [ ]:
# 10. Guardar el listado de compras en una nueva colección
pipeline = [
    {
        "$match": {
            "fecha_compra": datetime(2024, 4, 11)
        }
    },
    { "$unwind": "$productos" },
    { "$unwind": "$productos.proveedores" },
    { "$unwind": "$productos.proveedores.direcciones_almacenes" },
    {
        "$match": {
            "productos.proveedores.direcciones_almacenes.calle": "Avenida de Andalucía",
            "productos.proveedores.direcciones_almacenes.ciudad": "Sevilla",
            "productos.proveedores.direcciones_almacenes.location.coordinates": [-5.963709, 37.387533]
        }
    },
    {
        "$project": {
            "_id": 1,
            "cliente": "$cliente.nombre",
            "productos": "$productos.nombre",
            "precio_compra": 1,
            "fecha_compra": 1,
            "direccion_envio": 1,
            "almacen": {
                "direccion": "$productos.proveedores.direcciones_almacenes.calle",
                "ciudad": "$productos.proveedores.direcciones_almacenes.ciudad",
                "location": "$productos.proveedores.direcciones_almacenes.location"
            }
        }
    },
    {
        "$out": "compras_a_enviar"
    }
]

Compra.aggregate(pipeline)
